In [ ]:
!pip install mediapipe

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.5 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of opencv-contrib-python to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.6/35.6 MB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 109.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 26.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 10.0 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: opencv-contrib-python
    Found existing installation: opencv-contrib-python 4.12.0.88
    Uninstalling open

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download debashishsau/aslamerican-sign-language-aplhabet-dataset

Dataset URL: https://www.kaggle.com/datasets/debashishsau/aslamerican-sign-language-aplhabet-dataset
License(s): CC0-1.0
 99% 4.17G/4.20G [00:58<00:00, 37.6MB/s]
100% 4.20G/4.20G [00:58<00:00, 77.2MB/s]


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/aslamerican-sign-language-aplhabet-dataset.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [ ]:
!pip install split-folders

In [ ]:
import os
import splitfolders

input_dir = "/content/ASL_Alphabet_Dataset/asl_alphabet_train"
output_dir = '/content/split_data'

os.makedirs(output_dir, exist_ok=True)

input_dir_extracted='/content/ASL_Alphabet_Dataset/asl_alphabet_train'

if os.path.isdir(input_dir_extracted):
    splitfolders.ratio(input_dir_extracted, output=output_dir, seed=1337, ratio=(.50, .50), move=False)

Copying files: 223074 files [02:00, 1844.61 files/s]


In [ ]:
import os
import cv2
import numpy as np
import mediapipe as mp
from tqdm import tqdm
from multiprocessing import Pool, cpu_count
from datetime import datetime
import time

PADDING = 30
INPUT_DIR = "/content/split_data/train"
OUTPUT_DIR = "/content/cleaned_dataset"
NUM_WORKERS = min(6, cpu_count())
mp_hands = mp.solutions.hands

def process_image(task):
    try:
        class_name, img_name = task

        input_path = os.path.join(INPUT_DIR, class_name, img_name)
        output_class_path = os.path.join(OUTPUT_DIR, class_name)
        output_path = os.path.join(output_class_path, img_name)

        if os.path.exists(output_path):
            return

        img = cv2.imread(input_path, cv2.IMREAD_COLOR)
        if img is None:
            return

        h, w = img.shape[:2]
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        with mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.7) as hands:
            results = hands.process(img_rgb)

            if not results.multi_hand_landmarks:
                return

            lm = results.multi_hand_landmarks[0]
            x_coords = [int(pt.x * w) for pt in lm.landmark]
            y_coords = [int(pt.y * h) for pt in lm.landmark]

            x_min = max(min(x_coords) - PADDING, 0)
            y_min = max(min(y_coords) - PADDING, 0)
            x_max = min(max(x_coords) + PADDING, w)
            y_max = min(max(y_coords) + PADDING, h)

            hand_crop = img[y_min:y_max, x_min:x_max]

            if hand_crop.size == 0:
                return

            os.makedirs(output_class_path, exist_ok=True)
            cv2.imwrite(output_path, hand_crop)

    except Exception:
        return

def get_all_tasks():
    tasks = []
    for class_name in os.listdir(INPUT_DIR):
        class_path = os.path.join(INPUT_DIR, class_name)
        if not os.path.isdir(class_path):
            continue
        os.makedirs(os.path.join(OUTPUT_DIR, class_name), exist_ok=True)
        for img_name in os.listdir(class_path):
            tasks.append((class_name, img_name))
    return tasks

if __name__ == "__main__":
    all_tasks = get_all_tasks()
    total = len(all_tasks)

    start_time = time.time()
    start_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    print(f" Starting hand cropping")
    print(f"Start Time       : {start_str}")
    print(f"Processes Used   : {NUM_WORKERS}")
    print(f"Total Images     : {total}")

    with Pool(NUM_WORKERS) as pool:
        list(tqdm(pool.imap_unordered(process_image, all_tasks), total=total, desc="Processing"))

    end_time = time.time()
    end_str = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    elapsed = end_time - start_time
    mins, secs = divmod(elapsed, 60)

    print(f"⏳ Total Time       : {int(mins)} mins {int(secs)} secs")


In [ ]:
!zip -r cleaned_dataset.zip cleaned_dataset
files.download('cleaned_dataset.zip')


In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/cleaned_dataset.zip','r')
zip_ref.extractall('/content/data')
zip_ref.close()

In [ ]:
import os
import splitfolders

input_dir = "/content/data"
output_dir = '/content/final_train_test'

os.makedirs(output_dir, exist_ok=True)

input_dir_extracted='/content/data'

if os.path.isdir(input_dir_extracted):
    splitfolders.ratio(input_dir_extracted, output=output_dir, ratio=(.8, .2), move=False)

Copying files: 21070 files [00:02, 9080.29 files/s]


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout,Activation
from tensorflow.keras.layers import BatchNormalization

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    directory='/content/final_train_test/train',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256, 256),
    shuffle=True,

)
test_ds=tf.keras.utils.image_dataset_from_directory(
    directory='/content/final_train_test/val',
    labels='inferred',
    label_mode='categorical',
    batch_size=32,
    image_size=(256,256),
    shuffle=True,
)

Found 16848 files belonging to 28 classes.
Found 4222 files belonging to 28 classes.


In [ ]:
def process(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.rgb_to_grayscale(image)
    return image, label

train_ds=train_ds.map(process)
test_ds=test_ds.map(process)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization

model = Sequential()

model.add(Conv2D(32, (3,3), input_shape=(256,256,1), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(tf.keras.layers.Activation('relu'))

model.add(Conv2D(64, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(tf.keras.layers.Activation('relu'))

model.add(Conv2D(128, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(tf.keras.layers.Activation('relu'))

model.add(Conv2D(256, (3,3), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2,2)))
model.add(tf.keras.layers.Activation('relu'))

model.add(Flatten())

model.add(Dense(128))
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(Dropout(0.3))

model.add(Dense(28, activation='softmax'))


model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
model.fit(train_ds,epochs=10,validation_data=test_ds)

Epoch 1/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 216s 391ms/step - accuracy: 0.3821 - loss: 2.3141 - val_accuracy: 0.6978 - val_loss: 1.1230
Epoch 2/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 232s 344ms/step - accuracy: 0.8098 - loss: 0.7004 - val_accuracy: 0.7260 - val_loss: 1.0877
Epoch 3/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 200s 341ms/step - accuracy: 0.8922 - loss: 0.3825 - val_accuracy: 0.7189 - val_loss: 1.0026
Epoch 4/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 202s 341ms/step - accuracy: 0.9257 - loss: 0.2516 - val_accuracy: 0.8347 - val_loss: 0.5715
Epoch 5/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 201s 340ms/step - accuracy: 0.9453 - loss: 0.1894 - val_accuracy: 0.8887 - val_loss: 0.3935
Epoch 6/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 179s 339ms/step - accuracy: 0.9587 - loss: 0.1348 - val_accuracy: 0.6222 - val_loss: 1.5071
Epoch 7/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 180s 341ms/step - accuracy: 0.9589 - loss: 0.1317 - val_accuracy: 0.7873 - val_loss: 0.7037
Epoch 8/10
527/527 ━━━━━━━━━━━━━━━━━━━━ 202s 341ms/step - accuracy: 0.9707 -

In [ ]:
model.save("Hand_Alphabets18.h5")
from google.colab import files
files.download('Hand_Alphabets18.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import zipfile
zipref=zipfile.ZipFile('/content/cleaned_dataset.zip','r')
zipref.extractall('/content/dataset')
zipref.close()